In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

# 前処理制御用フラグ

In [ ]:
UNDER_SAMPLE=True
TARGET_ENCODE=True
VARIANCE_THRESH=True
DUMMIFICATION=False
POLYNOMIAL=True

# EDA

ref.
- https://www.kaggle.com/code/bertcarremans/data-preparation-exploration/notebook
- https://zg104.github.io/Safe_Driver_Prediction/#jump1

In [ ]:
train = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/test.csv')

## データの意味

- 類似のグループに属する素性は特徴名にタグ付けされる（例：ind, reg, car, calc）
- 特徴名には、2値素性を表すbinという接頭辞と、カテゴリ素性を表すcatという接尾辞が含まれる
- 接頭辞がない特徴名は連続または順序データとなる
- **-1**は欠損データ
- target列はその契約者にクレームが提出されたかどうかを意味する

In [ ]:
print('train.shape:', train.shape)
print('test.shape:', test.shape)

In [ ]:
# 重複データを削除
train.drop_duplicates()
test.drop_duplicates()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.head()

## 基礎統計量のチェック

In [ ]:
train.describe()

## メタデータの設定

- データ管理を容易化するため, DataFrameに以下の各特徴のメタデータを設定する

    - role(用途): input, ID, target
    - level(基準): nominal(平均), interval(間隔データ), ordinal(順序データ), binary
    - keep: True, False
    - dtype: int, float, str

In [ ]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'id':
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [ ]:
meta

In [ ]:
# メタデータ(level)が'nominal' かつ keepのindexを取り出す
meta[(meta.level == 'nominal') & (meta.keep)].index

In [ ]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

## 間隔変数の調査

- reg(ps_reg_01, ps_reg_02)
    - ps_reg_03のみ欠損値がある
    - 変数により範囲が異なるため、標準化の有無は使用したい分類器に依存して変化する

- car(ps_car_12, ..., ps_car_15)
    - 欠損値はps_car_12, ps_car_14のみ
    - 標準化の話はregと同じ

- calc(ps_calc_01, ps_calc_02, ps_calc_03)
    - いずれも欠損値は含まれない
    - 最大値は0.9っぽい
    - 3つの変数の分布はとても似ている


- 全般
    - 変数間の値の範囲はとても小さいので, 既に何らかのスケール変換(対数変換)が行われているかもしれない

In [ ]:
# 間隔データの基礎統計量を取得
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

## 順序変数の調査

- 欠損値があるのはps_car_11のみ
- スケーリングを適用することができそう?

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

## バイナリ変数の調査
- 訓練データのアプリオリ(事前確率)は3.645%であり、強くアンバランスである。(アプリオリ計算ってどうしたの?)
- 平均値から、ほとんどの変数で値がゼロであると結論づけることができる。

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

## 不均衡データの取り扱い

前述したように、target=1 のレコードの比率は target=0 よりもはるかに少ないので、精度は高いが実際には付加価値のないモデルになってしまうことがある。この問題に対処するために、2つの方策が考えられる。
- target=1のレコードをオーバーサンプリングする
- target=0のレコードをアンダーサンプリングする

もちろん、これ以外にも多くの戦略があり、MachineLearningMastery.comにはその概要が掲載されています。我々はかなり大きな学習セットを持っているので、アンダーサンプリングを行うことができます。

In [ ]:
desired_apriori=0.10

# Get the indices per target value
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target=0
# target=0のアンダーサンプリング率を計算し, 適用後のデータ数を計算する
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
# 期待するアプリオリになるように, taget=0となるレコードをランダム選択する
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
# 残されたデータ(target=1となるデータ)を加えて, リスト化する
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame

if UNDER_SAMPLE:
    train = train.loc[idx_list].reset_index(drop=True)

## データ品質のチェック

### 欠損データ

- ps_car_03_cat と ps_car_05_cat は、欠損値を持つレコードの割合が大きいため。これらの変数を削除する
    - その他のカテゴリー変数で欠損値があるものについては、欠損値-1のままでもよい。
- ps_reg_03 (連続) は、全レコードの 18% に欠損値がある。平均値で置き換える。
- その他は、欠損値が全レコードの10%未満です。平均値で置き換える。

- 欠損値が多すぎる変数を削除し、他の変数を平均値や最頻値でインピュテーションすることにします。

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        # print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        print(f'Variable {f} has {missings} records ({missings_perc:.2%}) with missing values')
        
print(f'In total, there are {len(vars_with_missing)} variables with missing values')

# カーディナリティの調査

- カーディナリティとは、変数に含まれる異なる値の数のことである。

- 後でカテゴリ変数からダミー変数を作成するので, 多くの異なる値を持つ変数があるかどうかを確認する必要があります。
- ダミー変数が多くなってしまうので、これらの変数を別扱いする必要がある.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

## ターゲットエンコーディング

In [ ]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
if TARGET_ENCODE:

    train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                                 test["ps_car_11_cat"], 
                                 target=train.target, 
                                 min_samples_leaf=100,
                                 smoothing=10,
                                 noise_level=0.01)

    train['ps_car_11_cat_te'] = train_encoded
    train.drop('ps_car_11_cat', axis=1, inplace=True)
    meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta
    test['ps_car_11_cat_te'] = test_encoded
    test.drop('ps_car_11_cat', axis=1, inplace=True)

## カテゴリ変数の調査

- カテゴリー値ごとにターゲット＝1の割合を算出
- 欠損値を持つ顧客は、保険金請求をする確率がかなり高い（場合によってはかなり低い）ように見える。

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize=(20,10))
    # Calculate the percentage of target=1 per category value
    
    cat_perc = train[[f, 'target']].groupby([f],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax=ax, x=f, y='target', data=cat_perc, order=cat_perc[f])
    plt.ylabel('% target', fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=18)
    plt.show();

## 間隔変数のチェック

区間変数間の相関を確認する。ヒートマップは、変数間の相関を可視化するよい方法である。
以下の変数間は強い相関がある

- ps_reg_02 and ps_reg_03: (0.7)
- ps_car_12 and ps_car13: (0.67)
- ps_car_12 and ps_car14: (0.58)
- ps_car_13 and ps_car15: (0.67)

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()

    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show();
    
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)

In [ ]:
s = train.sample(frac=0.1)

### ps_reg_02 and ps_reg_03

In [ ]:
sns.lmplot(x='ps_reg_02', y='ps_reg_03', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

### ps_car_12 and ps_car_13

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

### ps_car_12 and ps_car_14

In [ ]:
sns.lmplot(x='ps_car_12', y='ps_car_14', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_15', y='ps_car_13', data=s, hue='target', palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

相関のある変数のうち、どれを残すかをどうやって決めればいいのでしょうか？主成分分析(PCA)を実行して、次元を削減することができます。
AllState Claims Severity Competitionで、私はそれを行うためにこのカーネルを作りました。
しかし、相関のある変数の数はかなり少ないので、我々はモデルに力仕事をさせます。

# 特徴量エンジニアリング

## ダミー変数の作成

カテゴリ変数の値は、いかなる順序や大きさも表さない。例えば、カテゴリ2はカテゴリ1の2倍の値ではありません。したがって、我々はそれに対処するためにダミー変数を作成することができます。この情報は、元の変数のカテゴリに対して生成された他のダミー変数から得られるので、我々は最初のダミー変数を削除します。

In [ ]:
if DUMMIFICATION:
    v = meta[(meta.level == 'nominal') & (meta.keep)].index
    print('Before dummification we have {} variables in train'.format(train.shape[1]))
    train = pd.get_dummies(train, columns=v, drop_first=True)
    print('After dummification we have {} variables in train'.format(train.shape[1]))

## 相互特徴量の作成

相互作用の効果は、相互作用する特徴量の対応する値の積からなる新しい特徴量を含めることによって説明することができる。

In [ ]:
if POLYNOMIAL:
    v = meta[(meta.level == 'interval') & (meta.keep)].index
    poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
    interactions = pd.DataFrame(data=poly.fit_transform(train[v]), columns=poly.get_feature_names_out(v))
    interactions.drop(v, axis=1, inplace=True)  # Remove the original columns
    # Concat the interaction variables to the train data
    print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
    train = pd.concat([train, interactions], axis=1)
    print('After creating interactions we have {} variables in train'.format(train.shape[1]))

## 特徴選択

個人的には、どの特徴を残すかは、分類器のアルゴリズムに任せるのがいいと思います。
しかし、1つだけ私たち自身でできることがあります。それは、分散がない、あるいは非常に小さい特徴を削除することです。
Sklearnにはそれを行うための便利なメソッドがあります。VarianceThresholdです。
デフォルトでは、分散がゼロの素性を削除します。これまでのステップで分散ゼロの変数がないことを確認したので、これは今回のコンペティションには適用されないでしょう。
しかし、もし1%未満の分散を持つ特徴を削除すると、31個の変数を削除することになります。

### 分散が低い特徴量を削除

In [ ]:
if VARIANCE_THRESH:
    selector = VarianceThreshold(threshold=.01)
    selector.fit(train.drop(['id', 'target'], axis=1)) # Fit to train without id and target variables

    f = np.vectorize(lambda x : not x) # Function to toggle boolean array elements

    v = train.drop(['id', 'target'], axis=1).columns[f(selector.get_support())]
    print('{} variables have too low variance.'.format(len(v)))
    print('These variables are {}'.format(list(v)))

分散に基づいて選択すると、むしろ多くの変数を失うことになります。しかし、我々はそれほど多くの変数を持っていないので、分類器に選択させることにします。多くの変数があるデータセットでは、処理時間を短縮することができます。

# モデリング

## 評価メトリクスの定義

In [ ]:
# Define the gini metric - from https://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703#5897
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

## Validation(Stratified KFold)
ref. https://www.kaggle.com/code/sudosudoohio/stratified-kfold-xgboost-eda-tutorial-0-281/notebook#3.3-Stratified-KFold

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = 5
skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)

## XGBoost

In [ ]:
# More parameters has to be tuned. Good luck :)
params = {
    'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'eta': 0.025,
    'gamma': 0.65,
    'num_boost_round' : 700
    }

## 学習と推論

In [ ]:
X = train.drop(['id', 'target'], axis=1).values
y = train.target.values
test_id = test.id.values
test = test.drop('id', axis=1)

### Submissionファイルの作成

ref. https://www.kaggle.com/code/sudosudoohio/stratified-kfold-xgboost-eda-tutorial-0-281/notebook#Create-a-submission-file

In [ ]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)

In [ ]:
import xgboost as xgb

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_test = xgb.DMatrix(test.values)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    # Train the model! We pass in a max of 1,600 rounds (with early stopping after 70)
    # and the custom metric (maximize=True tells xgb that higher metric is better)
    mdl = xgb.train(params, d_train, 1600, watchlist, early_stopping_rounds=70, feval=gini_xgb, maximize=True, verbose_eval=100)

    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)
    sub['target'] += p_test/kfold

In [ ]:
sub.to_csv('StratifiedKFold.csv', index=False)